In [1]:
import numpy as np
from mindquantum.core.gates import RX, RY, RZ, H, X, Y, Z, CNOT
from mindquantum.core.circuit import Circuit
import mindspore as ms
from mindquantum.simulator import  Simulator
from mindquantum.core.gates import GroupedPauli
from mindquantum.core.operators import TimeEvolution,QubitOperator
from mindquantum.core.parameterresolver import PRConvertible,PRGenerator,ParameterResolver
from DQAS_tool import generate_pauli_string,one_hot
from mindquantum.core.gates import RotPauliString
from mindquantum.core.gates import UnivMathGate
from mindspore import Tensor, ops
from mindquantum.core.circuit import UN
from mindquantum.core.operators import Hamiltonian             # 导入Hamiltonian模块，用于构建哈密顿量
from mindquantum.framework import MQLayer
from mindspore.nn import  TrainOneStepCell
from mindspore.nn import SoftmaxCrossEntropyWithLogits                         # 导入SoftmaxCrossEntropyWithLogits模块，用于定义损失函数
from mindspore.nn import Adam                                                  # 导入Adam模块用于定义优化参数
from mindspore.train import Accuracy, Model, LossMonitor                       # 导入Accuracy模块，用于评估预测准确率
import mindspore as ms
from mindspore import Parameter, Tensor
from mindspore.dataset import NumpySlicesDataset
from torch.utils.data import DataLoader# 导入NumpySlicesDataset模块，用于创建模型可以识别的数据集
import sys
sys.path.append('..')
from data_processing import X_train,X_test,y_train,y_test
from mindquantum.algorithm.nisq import HardwareEfficientAnsatz   

from DQAS_tool import Mindspore_ansatz,loss_fn,vag_nnp,sampling_from_structure,vag_nnp,sampling_from_structure
from mindquantum.framework import MQOps
import mindspore.nn as nn
import numpy as np
import tensorcircuit as tc
import tensorflow as tf
  
num_layer = 6
# 定义标准差和形状
stddev = 0.02
shape_parametized = 12
shape_unparametized = 4
shape_nnp = (num_layer, shape_parametized)
shape_stp = (num_layer, shape_parametized+shape_unparametized)

shape_stp = (num_layer, shape_parametized)

rtype = np.float64
ctype = np.complex128
# 使用 numpy 生成随机数矩阵
np.random.seed(10)
nnp = np.random.normal(loc=0.0, scale=stddev, size=shape_nnp).astype(rtype)
stp = np.random.normal(loc=0.0, scale=stddev, size=shape_stp).astype(rtype)
# #Operator Pool
unbound_opeartor_pool = [generate_pauli_string(n=8,seed=i)[0] for i in range(shape_parametized)]
bound_opeartor_pool = [generate_pauli_string(n=8,seed=i)[1] for i in range(shape_parametized,shape_parametized+shape_unparametized)]

/opt/miniconda3/envs/Quantum/lib/python3.9/site-packages/mindquantum/simulator/__init__.py:17: UserWarning: Unable import mqvector gpu backend due to: cannot import name '_mq_vector_gpu' from partially initialized module 'mindquantum' (most likely due to a circular import) (/opt/miniconda3/envs/Quantum/lib/python3.9/site-packages/mindquantum/__init__.py)
  from .available_simulator import SUPPORTED_SIMULATOR
Please first ``pip install -U qiskit`` to enable related functionality in translation module
Please first ``pip install -U cirq`` to enable related functionality in translation module


In [2]:
import mindspore.numpy as mnp
def nmf_gradient(structures:np.array, oh:ms.Tensor,num_layer: int,size_pool:int):
    """
    使用 MindSpore 实现蒙特卡洛梯度计算。
    """
      # Step 1: 获取选择的索引
    choice = ops.Argmax(axis=-1)(oh)
    # Step 2: 计算概率
    softmax = ops.Softmax(axis=-1)
    prob = softmax(ms.Tensor(structures))
    # Step 3: 获取概率矩阵中的值
    indices = mnp.stack((mnp.arange(num_layer, dtype=ms.int64), choice), axis=1)
    prob = ops.GatherNd()(prob, indices)
    # Step 4: 变换概率矩阵
    prob = prob.reshape(-1, 1)
    prob = ops.Tile()(prob, (1, size_pool))
    
    # Step 5: 生成蒙特卡洛梯度
    gradient = ops.TensorScatterAdd()(Tensor(-prob, ms.float64), indices, mnp.ones((num_layer,), dtype=ms.float64))
    return gradient
    
    
# 对向量化版本的封装
# nmf_gradient_vmap = ops.vmap(nmf_gradient, in_axes=(None, 0, None, None))

def best_from_structure(structures: np.array)->Tensor:
    return ops.Argmax(axis=-1)(ms.Tensor(structures))

In [3]:
unbound_opeartor_pool

['I0 I1 X2 Z3 I4 I5 Z6 I7',
 'Y0 X1 Z2 X3 I4 I5 I6 I7',
 'X0 X1 X2 Z3 Y4 Z5 Z6 Y7',
 'Y0 Y1 Z2 I3 X4 X5 I6 Z7',
 'Y0 Z1 X2 I3 I4 Y5 X6 X7',
 'Z0 Z1 X2 I3 Y4 X5 Y6 X7',
 'X0 I1 Z2 X3 X4 Y5 I6 Z7',
 'Z0 Y1 I2 X3 X4 X5 Z6 X7',
 'Y0 Z1 I2 Y3 Y4 X5 X6 Y7',
 'I0 Z1 Z2 Y3 Y4 X5 Z6 I7',
 'X0 I1 I2 X3 Y4 I5 I6 Z7',
 'I0 I1 I2 Y3 Y4 I5 Y6 X7']

In [4]:
shape_stp

(6, 12)

In [ ]:
import numpy as np
import tensorcircuit as tc
import tensorflow as tf
from DQAS_tool import  DQASAnsatz_from_result
K = tc.set_backend("tensorflow")
lr = tf.keras.optimizers.schedules.ExponentialDecay(0.06, 100, 0.5)
structure_opt = tc.backend.optimizer(tf.keras.optimizers.Adam(0.05))
network_opt = tc.backend.optimizer(tf.keras.optimizers.Adam(lr))
verbose = True
# 设置超参数
epochs = 100
batch_size=50
shape_nnp = (num_layer, shape_parametized)
shape_stp = (num_layer, shape_parametized)
nnp = np.random.normal(loc=0.0, scale=stddev, size=shape_nnp).astype(rtype)
stp = np.random.normal(loc=0.0, scale=stddev, size=shape_stp).astype(rtype)
#print(stp.shape)
avcost1 = 0

ops_onehot = ops.OneHot(axis=-1)

batch_loss_history=[] # 记录每个epoch的batch_size损失值
structure_distribution_history=[] # 记录每个epoch的结构参数
ansatz_params_history=[] # 记录每个epoch的网络参数
best_candidates_history=[] # 记录每个epoch的最佳候选


for epoch in range(epochs):  # 更新结构参数的迭代
    avcost2 = avcost1
    costl = []
    tmp = np.stack([sampling_from_structure(stp,num_layer,shape_parametized) for _ in range(batch_size)])
    batch_structure = ops_onehot(ms.Tensor(tmp),shape_parametized,ms.Tensor(1),ms.Tensor(0))
    #print(batch_structure.shape)
    # print(tmp,batch_structure)
    loss_value = []
    grad_nnps = []
    grad_stps = []
    
    for i in batch_structure:          
        infd, grad_nnp = vag_nnp(Structure_params=i,Ansatz_params=nnp,paramerterized_pool=unbound_opeartor_pool,num_layer=num_layer,n_qbits=8)(ms.Tensor(X_train),ms.Tensor(y_train))
        gs = nmf_gradient(structures=stp,oh=i,num_layer=num_layer,size_pool=shape_parametized)
        #print(infd,grad_nnp)
        loss_value.append(infd)
        grad_nnps.append(grad_nnp[0])
        grad_stps.append(gs)
    
    infd = ops.stack(loss_value)
    gnnp = ops.addn(grad_nnps)
    gstp = [(infd[i] - avcost2) * grad_stps[i] for i in range(infd.shape[0])]
    gstp_averge = ops.addn(gstp) / infd.shape[0]
    avcost1 = sum(infd) / infd.shape[0]
    
    gnnp_tf = tf.convert_to_tensor(gnnp.reshape(nnp.shape).asnumpy(),dtype=tf.float64)
    nnp_tf = tf.convert_to_tensor(nnp,dtype=tf.float64)
    gstp_averge_tf = tf.convert_to_tensor(gstp_averge.reshape(stp.shape).asnumpy(),dtype=tf.float64)
    stp_tf = tf.convert_to_tensor(stp,dtype=tf.float64)
    # 更新参数
    nnp_tf = network_opt.update(gnnp_tf, nnp_tf)
    stp_tf = structure_opt.update(gstp_averge_tf, stp_tf) 
    
    nnp = nnp_tf.numpy()
    stp = stp_tf.numpy()
    
    batch_loss_history.append(avcost1)
    structure_distribution_history.append(stp)
    ansatz_params_history.append(nnp)
    best_candidates_history.append(best_from_structure(cand_preset.asnumpy()))
    
    
    
    if epoch % 2 == 0 or epoch == epochs - 1:
        print("----------epoch %s-----------" % epoch)
        print(
            "batched average loss: ",
            avcost1,
        )
    
        if verbose:
            print(
                "strcuture parameter: \n",
                stp,
                "\n network parameter: \n",
                nnp,
            )
        
        cand_preset = best_from_structure(stp)
        print("best candidates so far:",cand_preset)
        # print(
        #     "corresponding weights for each gate:",
        #     [K.numpy(nnp[j, i]) if i < 6 else 0.0 for j, i in enumerate(cand_preset)],
        # )
    # if epoch % 10 == 0 or epoch == epochs - 1:
    #     _,acc = DQASAnsatz_from_result(best_candidate=cand_preset.asnumpy(),parameterized_pool=unbound_opeartor_pool,num_layer=num_layer,n_qbits=8)
    #     print("目前的二分类精度是:",np.max(acc))

----------epoch 0-----------
batched average loss:  0.9081669
strcuture parameter: 
 [[-0.15093987 -0.09307831 -0.10991064  0.14375304  0.10871103  0.13696269
   0.13373923 -0.1506358   0.12297179 -0.11367219 -0.13147277  0.11278609]
 [-0.11126413 -0.12073264  0.09988689 -0.10814521  0.12454894  0.12485383
  -0.11805901 -0.11099384 -0.1256943   0.10261504 -0.13482433  0.14210409]
 [ 0.07962673 -0.10917706 -0.14883385 -0.15216886 -0.14011925  0.1147135
   0.13459126  0.08601307  0.15346261  0.14325558  0.11721266  0.11418458]
 [-0.13904486 -0.10823294  0.12135728  0.14823083  0.10625695  0.13095147
  -0.12071948  0.10304922 -0.08192812  0.1254643   0.13227715 -0.14135917]
 [-0.13442656  0.12166407  0.10818468 -0.10795138  0.12874859  0.10434269
  -0.11611543 -0.11991133  0.11669978 -0.14296807  0.10327111  0.12420161]
 [ 0.14015985  0.10047664 -0.09563024  0.12353309  0.0945932   0.12646042
   0.13759572  0.12527749 -0.07359132 -0.11381439 -0.10212258 -0.11977328]] 
 network parameter: 

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x1039326d0>>
Traceback (most recent call last):
  File "/opt/miniconda3/envs/Quantum/lib/python3.9/site-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 
